In [1]:
# Logger
import logging
logging.basicConfig(level=logging.WARN)

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

# Load BACS module
from bacs.agents.bacs import BACS, Configuration
from bacs.agents.bacs.utils.GymMazeWrapper import _maze_metrics, parse_metrics_to_df, plot_performance

# Load environments
import gym
import my_mazes

# Allow to parallelize all benchmarks to do
import parmap

# For calculation standard deviation
import statistics

# Agent - BACS - BENCHMARKING

## Main Parameters

In [2]:
CLASSIFIER_LENGTH = 8
NUMBER_OF_POSSIBLE_ACTIONS = 8

NUMBER_OF_EXPLORE_TRIALS = 1000
DO_ACTION_PLANNING_EXPLORE = False
DO_GA_EXPLORE = False
DO_SUBSUMPTION_EXPLORE = True
BETA_EXPLORE = 0.05
EPSILON = 0.8

NUMBER_OF_EXPLOIT_TRIALS = 500
DO_ACTION_PLANNING_EXPLOIT = False
DO_SUBSUMPTION_EXPLOIT = True
BETA_EXPLOIT = 0.00

LENGTH_OF_BEHAVIORAL_SEQUENCES = 1

NUMBER_OF_ITERATIONS_TO_BENCH = 30

NB_OF_PROCESSES = 12

## Full Details of Agent Parameters

In [3]:
cfg_explore = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=1,
    do_ga=DO_GA_EXPLORE,
    do_subsumption=DO_SUBSUMPTION_EXPLORE,
    do_action_planning=DO_ACTION_PLANNING_EXPLORE,
    action_planning_frequency=50,
    beta=BETA_EXPLORE,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=EPSILON,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20,
    theta_ga=100,
    theta_as=20,
    mu=0.3,
    chi=0.8,
    bs_max=LENGTH_OF_BEHAVIORAL_SEQUENCES
)

cfg_exploit = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=1,
    do_ga=False,
    do_subsumption=DO_SUBSUMPTION_EXPLOIT,
    do_action_planning=DO_ACTION_PLANNING_EXPLOIT,
    action_planning_frequency=50,
    beta=BETA_EXPLOIT,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=0.0,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20,
    theta_ga=100,
    theta_as=20,
    mu=0.3,
    chi=0.8,
    bs_max=LENGTH_OF_BEHAVIORAL_SEQUENCES
)

### Benchmarking - Maze

In [4]:
def bench_on_maze(env,n):
    avg_explore_list = []
    avg_exploit_list = []
    knowledge_list = []
    population_list = []
    numerosity_list = []
    reliable_list = []
    
    for i in range(n):
        
        # Initialize environment
        maze = gym.make(env.id)
        
        # Reset it, by putting an agent into random position
        situation = maze.reset()
    
        # Training of BACS - Exploration
        agent_explore = BACS(cfg_explore)
        population_explore, metrics_explore = agent_explore.explore(maze, NUMBER_OF_EXPLORE_TRIALS)
    
        # Using BACS - Exploitation
        agent_exploit = BACS(cfg_exploit, population_explore)
        agent_exploit.clean_population()
        population_exploit, metrics_exploit = agent_exploit.exploit(maze, NUMBER_OF_EXPLOIT_TRIALS)

        # Plotting performance
        #metrics_df = parse_metrics_to_df(metrics_explore, metrics_exploit)
        #plot_performance(agent_exploit, maze, metrics_df, cfg_exploit, env.id)
        
        # Get average in exploration
        avg_step_explore = 0
        for trial in metrics_explore:
            avg_step_explore += trial['steps_in_trial']
        avg_step_explore /= NUMBER_OF_EXPLORE_TRIALS
        avg_explore_list.append(avg_step_explore)
        
        # Get average in exploitation
        avg_step_exploit = 0
        for trial in metrics_exploit:
            avg_step_exploit += trial['steps_in_trial']
        avg_step_exploit /= NUMBER_OF_EXPLOIT_TRIALS
        avg_exploit_list.append(avg_step_exploit)
        
        # Get other metrics
        knowledge_list.append(metrics_exploit[-1]['knowledge'])
        population_list.append(metrics_exploit[-1]['population'])
        numerosity_list.append(metrics_exploit[-1]['numerosity'])
        reliable_list.append(metrics_exploit[-1]['reliable'])
        
        print(env.id,'|','#' * (i+1) ,' ' * (n-(i+1)), '|')
    
    # Compute the means and standard deviations
    avg_explore = float(sum(avg_explore_list)) / n
    std_explore = statistics.pstdev(avg_explore_list)
    avg_exploit = float(sum(avg_exploit_list)) / n
    std_exploit = statistics.pstdev(avg_exploit_list)
    avg_knowledge = float(sum(knowledge_list)) / n
    std_knowledge = statistics.pstdev(knowledge_list)
    avg_population = float(sum(population_list)) / n
    std_population = statistics.pstdev(population_list)
    avg_numerosity = float(sum(numerosity_list)) / n
    std_numerosity = statistics.pstdev(numerosity_list)
    avg_reliable = float(sum(reliable_list)) / n
    std_reliable = statistics.pstdev(reliable_list)
    
    result = {
        'maze'             : env.id,
        'avg_explore'      : avg_explore,
        'std_explore'      : std_explore,
        'avg_exploit'      : avg_exploit,
        'std_exploit'      : std_exploit,
        'avg_knowledge'    : avg_knowledge,
        'std_knowledge'    : std_knowledge,
        'avg_population'   : avg_population,
        'std_population'   : std_population,
        'avg_numerosity'   : avg_numerosity,
        'std_numerosity'   : std_numerosity,
        'avg_reliable'     : avg_reliable,
        'std_reliable'     : std_reliable,
        'avg_exploit_list' : avg_exploit_list
    }
    
    print(result)
    
    return result

Custom functions for getting available environments in Gym depending on the type of mazes :

In [5]:
filter_envs_typeIII = lambda env: env.id.startswith("Maze10-") or env.id.startswith("MazeE1") \
    or env.id.startswith("MazeE2") or env.id.startswith("Woods10")

filter_envs_typeII = lambda env: env.id.startswith("MazeF4") or env.id.startswith("Maze7") \
    or env.id.startswith("MiyazakiB")

filter_envs_typeI = lambda env: env.id.startswith("MazeB") or env.id.startswith("MazeD") \
    or env.id.startswith("Littman") or env.id.startswith("MiyazakiA") \
    or env.id.startswith("Cassandra")

filter_envs_na = lambda env: env.id.startswith("MazeF1") or env.id.startswith("MazeF2") \
    or env.id.startswith("MazeF3") or env.id.startswith("Woods1-") \
    or env.id.startswith("Woods14") or env.id.startswith("Maze4") \
    or env.id.startswith("Maze5") or env.id.startswith("MazeA")

all_envs = [env for env in gym.envs.registry.all()]

Benchmarking without the genetic algorithms :

In [6]:
maze_envs = [env for env in all_envs if filter_envs_na(env) or filter_envs_typeI(env) or filter_envs_typeII(env) or filter_envs_typeIII(env)]

results = parmap.map(bench_on_maze, maze_envs, NUMBER_OF_ITERATIONS_TO_BENCH, pm_pbar=True, pm_processes=NB_OF_PROCESSES)

  0%|          | 0/24 [00:00<?, ?it/s]

Maze7-v0 | #                               |
MazeF4-v0 | #                               |
Littman57-v0 | #                               |
Maze7-v0 | ##                              |
MazeF4-v0 | ##                              |
Littman57-v0 | ##                              |
Cassandra4x4-v0 | #                               |
Maze10-v0 | #                               |
Littman89-v0 | #                               |
Maze7-v0 | ###                             |
MazeF4-v0 | ###                             |
MazeB-v0 | #                               |
Littman57-v0 | ###                             |
MazeD-v0 | #                               |
MazeA-v0 | #                               |
Maze7-v0 | ####                            |
Maze4-v0 | #                               |
MazeF4-v0 | ####                            |
Maze10-v0 | ##                              |
Littman57-v0 | ####                            |
Maze7-v0 | #####                           |
MazeF4-v0 | #####     

  4%|▍         | 1/24 [27:22<10:29:28, 1642.12s/it]

Littman89-v0 | #########                       |
Maze10-v0 | ############                    |
MazeB-v0 | ######                          |
Littman57-v0 | #############################   |
MazeF4-v0 | ############################    |
Littman57-v0 | ##############################  |
{'maze': 'Littman57-v0', 'avg_explore': 21.8731, 'std_explore': 1.5560381604146687, 'avg_exploit': 15.716466666666664, 'std_exploit': 20.79664434267114, 'avg_knowledge': 90.32520325203247, 'std_knowledge': 0.43781827700280496, 'avg_population': 56.43333333333333, 'std_population': 1.2827920936595905, 'avg_numerosity': 56.43333333333333, 'std_numerosity': 1.2827920936595905, 'avg_reliable': 31.433333333333334, 'std_reliable': 0.6674994798166929, 'avg_exploit_list': [4.076, 4.518, 4.172, 3.924, 43.55, 58.342, 3.844, 3.902, 4.052, 5.302, 4.148, 56.106, 4.124, 4.92, 56.212, 57.154, 6.218, 4.456, 4.054, 4.384, 3.828, 4.406, 4.064, 5.382, 55.442, 44.656, 3.912, 4.126, 4.116, 4.104]}


  8%|▊         | 2/24 [28:30<7:08:57, 1169.89s/it] 

...

 92%|█████████▏| 22/24 [4:25:32<47:38, 1429.26s/it]  

MazeE1-v0 | #############################   |
MazeE2-v0 | #########################       |
MazeE1-v0 | ##############################  |
{'maze': 'MazeE1-v0', 'avg_explore': 24.2944, 'std_explore': 1.7515705067167575, 'avg_exploit': 4.575799999999999, 'std_exploit': 2.2084328289536, 'avg_knowledge': 68.7361111111111, 'std_knowledge': 3.8422049112777574, 'avg_population': 1271.5333333333333, 'std_population': 84.72730112281138, 'avg_numerosity': 1271.5333333333333, 'std_numerosity': 84.72730112281138, 'avg_reliable': 272.23333333333335, 'std_reliable': 26.376357258389987, 'avg_exploit_list': [3.174, 3.73, 3.03, 7.058, 9.88, 3.262, 8.424, 3.222, 4.458, 3.366, 3.462, 3.144, 3.228, 3.17, 9.234, 3.276, 3.244, 8.21, 3.158, 3.114, 8.314, 3.164, 3.11, 6.56, 3.204, 3.154, 3.02, 7.1, 3.42, 3.384]}


 96%|█████████▌| 23/24 [4:32:52<18:52, 1132.49s/it]

MazeE2-v0 | ##########################      |
MazeE2-v0 | ###########################     |
MazeE2-v0 | ############################    |
MazeE2-v0 | #############################   |
MazeE2-v0 | ##############################  |
{'maze': 'MazeE2-v0', 'avg_explore': 29.033499999999993, 'std_explore': 1.5643535778930115, 'avg_exploit': 20.958399999999997, 'std_exploit': 15.609161330449501, 'avg_knowledge': 50.83333333333334, 'std_knowledge': 3.281496174572894, 'avg_population': 986.0, 'std_population': 69.19922928279861, 'avg_numerosity': 986.0, 'std_numerosity': 69.19922928279861, 'avg_reliable': 141.0, 'std_reliable': 13.147876888177295, 'avg_exploit_list': [4.814, 8.028, 6.988, 31.334, 4.996, 64.668, 9.918, 5.65, 22.848, 37.148, 12.04, 5.968, 32.904, 26.994, 31.972, 38.97, 36.222, 11.204, 14.246, 10.494, 11.842, 33.508, 6.602, 56.198, 17.758, 36.958, 5.498, 11.164, 20.324, 11.494]}


100%|██████████| 24/24 [5:09:08<00:00, 772.87s/it] 


Parsing the previous result to get a markdown string for each environment :

In [7]:
markdown_str = ''

for item in results:
    markdown_str += '|' + item['maze'] + '|'
    markdown_str += "{:.2f}".format(item['avg_explore']) + '|'
    markdown_str += "{:.2f}".format(item['std_explore']) + '|'
    markdown_str += "{:.2f}".format(item['avg_exploit']) + '|'
    markdown_str += "{:.2f}".format(item['std_exploit']) + '|'
    markdown_str += "{:.2f}".format(min(item['avg_exploit_list'])) + '|'
    markdown_str += "{:.2f}".format(max(item['avg_exploit_list'])) + '|'
    markdown_str += '0' + '|'
    markdown_str += "{:.2f}".format(item['avg_knowledge']) + '|'
    markdown_str += "{:.2f}".format(item['std_knowledge']) + '|'
    markdown_str += "{:.2f}".format(item['avg_population']) + '|'
    markdown_str += "{:.2f}".format(item['std_population']) + '|'
    markdown_str += "{:.2f}".format(item['avg_reliable']) + '|'
    markdown_str += "{:.2f}".format(item['std_reliable']) + '|'
    markdown_str += '\n'
    
print(markdown_str)

|Cassandra4x4-v0|18.23|3.07|2.85|0.24|2.25|3.26|0|81.89|2.02|429.03|39.42|127.30|10.03|
|Littman57-v0|21.87|1.56|15.72|20.80|3.83|58.34|0|90.33|0.44|56.43|1.28|31.43|0.67|
|Littman89-v0|29.39|2.57|4.25|0.21|3.88|4.74|0|100.00|0.00|204.73|13.01|143.87|2.54|
|MazeA-v0|48.79|1.67|4.23|0.08|4.06|4.33|0|100.00|0.00|107.43|3.71|107.43|3.71|
|MazeB-v0|34.42|2.59|5.17|4.22|3.64|21.45|0|100.00|0.00|281.60|16.67|184.07|8.12|
|MazeD-v0|27.41|2.01|2.90|0.10|2.72|3.12|0|99.97|0.19|229.77|14.11|180.47|5.87|
|MazeF4-v0|29.51|2.21|4.51|0.08|4.32|4.69|0|100.00|0.00|45.60|5.91|33.30|1.27|
|Maze4-v0|31.59|1.91|3.50|0.06|3.34|3.61|0|100.00|0.00|172.03|6.33|172.03|6.33|
|Maze5-v0|47.80|2.62|4.64|0.11|4.47|5.01|0|100.00|0.00|226.90|8.47|226.90|8.47|
|Maze7-v0|26.21|1.51|4.34|0.08|4.16|4.53|0|100.00|0.00|43.17|3.61|34.17|1.37|
|Maze10-v0|50.19|3.08|65.51|14.54|48.17|90.41|0|90.43|1.06|135.43|6.54|47.40|1.65|
|MazeE1-v0|24.29|1.75|4.58|2.21|3.02|9.88|0|68.74|3.84|1271.53|84.73|272.23|26.38|
|MazeE2-v0|29.03|1

### Last results
 
The parameters used are:
 
- no use of action planning
- use of subsumption in the anticipatory learning process
- use of subsumption in the genetic algorithm if this last one is used
- $\gamma$ = 0.95
- $\theta_i$ = 0.1
- $\theta_r$ = 0.9
- $u_{max}$ = 8 (length of the classifier)
- $\theta_{exp}$ = 20
- $\beta$ = 0.05
- $\epsilon$ = 0.8
- $bs_{max} = 1$
 
If genetic algorithms are used:

- $\theta_{ga}$ = 100
- $\theta_{as}$ = 20
- $\mu$ = 0.3
- $\chi$ = 0.8
 
The mazes in the following table have been sorted by aliasing type and then by complexity top-down.

#### Experiments with all environments without GA

For 1000 trials in Exploration and 500 trials in Exploitation for all environments, computed 30 times for calculating all averages, having $bs_{max} = 1$:
 
| | Exploration Avg | Exploration Std | Exploration Avg | Exploration Std | Best Exploration | Worst Exploration | Successful tries | Knowledge Avg | Knowledge Std | Population Avg | Population Std | Reliable Avg | Reliable Std |
|------------|:-----:|:-----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|**Type III**     ||||||||||||||
|MazeE2-v0|29.03|1.56|20.96|15.61|4.81|64.67|0|50.83|3.28|986.00|69.20|141.00|13.15|
|Woods101demi-v0|28.36|1.24|15.25|7.64|3.05|25.99|0|90.48|0.00|158.63|15.72|76.53|1.67|
|Maze10-v0|50.19|3.08|65.51|14.54|48.17|90.41|0|90.43|1.06|135.43|6.54|47.40|1.65|
|Woods102-v0|30.01|2.10|7.33|4.65|3.48|18.75|0|93.25|1.47|241.77|5.37|131.77|2.35|
|Woods100-v0|10.84|0.84|2.35|0.05|2.25|2.45|0|100.00|0.00|15.37|0.84|10.00|0.00|
|Woods101-v0|24.14|1.06|3.05|0.09|2.87|3.26|0|92.59|0.00|64.30|4.66|48.17|1.37|
|MazeE1-v0|24.29|1.75|4.58|2.21|3.02|9.88|0|68.74|3.84|1271.53|84.73|272.23|26.38|
|**Type II**      ||||||||||||||
|MazeF4-v0|29.51|2.21|4.51|0.08|4.32|4.69|0|100.00|0.00|45.60|5.91|33.30|1.27|
|Maze7-v0|26.21|1.51|4.34|0.08|4.16|4.53|0|100.00|0.00|43.17|3.61|34.17|1.37|
|MiyazakiB-v0|40.15|2.57|3.81|0.26|3.35|4.43|0|99.93|0.21|513.27|39.03|288.40|13.88|
|**Type I**       ||||||||||||||
|MazeB-v0|34.42|2.59|5.17|4.22|3.64|21.45|0|100.00|0.00|281.60|16.67|184.07|8.12|
|Littman89-v0|29.39|2.57|4.25|0.21|3.88|4.74|0|100.00|0.00|204.73|13.01|143.87|2.54|
|MiyazakiA-v0|29.12|3.04|3.42|0.20|3.10|4.00|0|96.33|2.18|701.10|56.58|324.03|18.73|
|MazeD-v0|27.41|2.01|2.90|0.10|2.72|3.12|0|99.97|0.19|229.77|14.11|180.47|5.87|
|Cassandra4x4-v0|18.23|3.07|2.85|0.24|2.25|3.26|0|81.89|2.02|429.03|39.42|127.30|10.03|
|Littman57-v0|21.87|1.56|15.72|20.80|3.83|58.34|0|90.33|0.44|56.43|1.28|31.43|0.67|
|**Not Aliased**||||||||||||||
|Maze4-v0|31.59|1.91|3.50|0.06|3.34|3.61|0|100.00|0.00|172.03|6.33|172.03|6.33|
|Maze5-v0|47.80|2.62|4.64|0.11|4.47|5.01|0|100.00|0.00|226.90|8.47|226.90|8.47|
|MazeA-v0|48.79|1.67|4.23|0.08|4.06|4.33|0|100.00|0.00|107.43|3.71|107.43|3.71|
|MazeF1-v0|11.32|0.41|1.79|0.03|1.73|1.87|0|100.00|0.00|14.03|0.18|14.03|0.18|
|MazeF2-v0|19.86|0.57|2.51|0.05|2.42|2.62|0|100.00|0.00|18.07|0.25|18.07|0.25|
|MazeF3-v0|29.59|1.05|3.38|0.06|3.25|3.48|0|100.00|0.00|28.70|1.10|28.70|1.10|
|Woods1-v0|9.44|0.43|1.63|0.02|1.60|1.67|0|100.00|0.00|54.77|1.75|54.77|1.75|
|Woods14-v0|70.44|1.76|9.54|0.20|9.11|10.02|0|100.00|0.00|39.17|2.37|39.17|2.37|


For 1000 trials in Exploration and 500 trials in Exploitation for all environments, computed 30 times for calculating all averages, having $bs_{max} = 2$:
 
| | Exploration Avg | Exploration Std | Exploration Avg | Exploration Std | Best Exploration | Worst Exploration | Successful tries | Knowledge Avg | Knowledge Std | Population Avg | Population Std | Reliable Avg | Reliable Std |
|------------|:-----:|:-----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|**Type III**     ||||||||||||||
|MazeE2-v0|32.44|2.31|15.89|11.31|4.42|38.55|0|42.26|2.77|1425.27|119.64|110.37|11.00|
|Woods101demi-v0|29.55|1.16|14.04|6.87|3.09|25.29|0|90.48|0.00|203.13|13.07|76.03|1.96|
|Maze10-v0|38.85|3.91|25.48|24.57|5.63|70.88|0|94.82|1.05|175.73|10.19|54.83|5.77|
|Woods102-v0|30.72|2.22|8.11|3.71|3.61|14.04|0|95.12|0.00|317.07|11.42|167.90|8.24|
|Woods100-v0|11.03|0.72|2.35|0.05|2.21|2.45|0|100.00|0.00|15.33|0.65|10.00|0.00|
|Woods101-v0|23.73|1.14|3.03|0.07|2.93|3.23|0|92.59|0.00|64.70|3.89|48.27|1.55|
|MazeE1-v0|27.45|1.87|4.84|2.87|2.93|14.61|0|64.58|4.56|1674.23|118.70|269.57|37.34|
|**Type II**      ||||||||||||||
|MazeF4-v0|29.27|1.70|4.49|0.08|4.35|4.65|0|100.00|0.00|43.83|4.63|33.00|1.37|
|Maze7-v0|27.04|2.07|4.32|0.11|4.01|4.52|0|100.00|0.00|42.80|2.54|33.83|1.04|
|MiyazakiB-v0|41.20|4.27|3.91|0.29|3.46|4.79|0|98.76|1.44|578.97|45.57|295.07|12.60|
|**Type I**       ||||||||||||||
|MazeB-v0|35.14|2.36|4.44|2.05|3.65|15.45|0|100.00|0.00|288.17|16.09|185.73|5.61|
|Littman89-v0|29.72|2.18|4.37|0.26|4.02|5.20|0|100.00|0.00|215.07|14.55|147.90|4.86|
|MiyazakiA-v0|29.43|3.30|3.37|0.19|3.10|3.84|0|93.02|3.05|844.37|60.48|319.57|27.56|
|MazeD-v0|27.43|2.80|2.89|0.12|2.62|3.12|0|99.97|0.19|230.13|16.24|181.83|6.72|
|Cassandra4x4-v0|21.32|3.48|3.03|0.34|2.55|4.02|0|85.47|1.96|603.40|61.62|150.80|15.43|
|Littman57-v0|22.39|1.75|10.24|16.72|3.56|63.31|0|90.24|0.00|55.90|1.37|31.37|0.71|
|**Not Aliased**||||||||||||||
|Maze4-v0|32.58|1.60|3.49|0.06|3.38|3.59|0|100.00|0.00|172.13|6.23|172.13|6.23|
|Maze5-v0|46.91|2.35|4.62|0.10|4.43|4.92|0|100.00|0.00|228.73|8.83|228.57|8.71|
|MazeA-v0|48.79|1.60|4.27|0.12|4.09|4.79|0|100.00|0.00|106.73|4.34|106.73|4.34|
|MazeF1-v0|11.38|0.29|1.80|0.03|1.74|1.87|0|100.00|0.00|14.00|0.00|14.00|0.00|
|MazeF2-v0|19.74|0.62|2.50|0.04|2.43|2.58|0|100.00|0.00|18.10|0.30|18.10|0.30|
|MazeF3-v0|29.59|1.09|3.36|0.05|3.24|3.47|0|100.00|0.00|28.67|0.83|28.67|0.83|
|Woods1-v0|9.39|0.42|1.63|0.02|1.57|1.68|0|100.00|0.00|54.57|1.28|54.57|1.28|
|Woods14-v0|70.34|1.70|9.46|0.23|9.02|9.98|0|100.00|0.00|38.30|2.19|38.30|2.19|


#### Experiments with type-III aliased mazes with more trials in exploration without GA

For 5000 trials in Exploration and 500 trials in Exploitation for type-III aliased environments, computed 30 times for calculating all averages, having $bs_{max} = 1$
 
| | Exploration Avg | Exploration Std | Exploration Avg | Exploration Std | Best Exploration | Worst Exploration | Successful tries | Knowledge Avg | Knowledge Std | Population Avg | Population Std | Reliable Avg | Reliable Std |
|------------|:-----:|:-----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|**Type III**     ||||||||||||||
|MazeE2-v0|29.37|1.58|16.82|22.32|4.45|67.60|0|72.17|1.17|2305.57|105.28|422.57|26.19|
|Woods101demi-v0|27.43|0.64|14.49|5.30|3.01|23.68|0|90.56|0.43|324.10|19.44|83.97|2.64|
|Maze10-v0|45.13|1.62|61.56|12.52|49.88|92.67|0|92.91|1.85|218.37|20.67|53.83|6.81|
|Woods102-v0|27.75|0.53|5.84|4.73|3.28|24.75|0|93.33|1.12|343.47|17.45|135.57|1.63|
|Woods100-v0|9.91|0.13|2.32|0.05|2.22|2.42|0|100.00|0.00|23.97|1.99|10.03|0.18|
|Woods101-v0|36.77|2.14|4.15|0.46|3.50|4.75|0|90.62|2.08|85.77|19.27|43.53|1.41|
|MazeE1-v0|26.81|1.11|5.04|3.59|3.28|19.35|0|93.88|1.25|2594.33|153.27|595.27|24.65|

For 5000 trials in Exploration and 500 trials in Exploitation for type-III aliased environments, computed 30 times for calculating all averages, having $bs_{max} = 2$
 
| | Exploration Avg | Exploration Std | Exploration Avg | Exploration Std | Best Exploration | Worst Exploration | Successful tries | Knowledge Avg | Knowledge Std | Population Avg | Population Std | Reliable Avg | Reliable Std |
|------------|:-----:|:-----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|**Type III**     ||||||||||||||
|MazeE2-v0|33.92|3.12|21.46|22.55|4.65|64.97|0|74.56|1.96|3533.47|206.84|475.23|30.90|
|Woods101demi-v0|27.42|0.70|15.93|6.36|3.11|25.01|0|90.56|0.43|435.03|25.61|89.40|2.86|
|Maze10-v0|42.22|2.44|57.32|14.23|9.20|89.12|0|95.46|0.91|366.37|28.54|65.17|3.70|
|Woods102-v0|28.49|1.06|7.45|7.84|3.58|41.71|0|95.16|0.22|457.57|26.52|184.60|2.48|
|Woods100-v0|11.35|0.16|2.33|0.05|2.25|2.47|0|100.00|0.00|31.90|2.89|10.03|0.18|
|Woods101-v0|37.43|2.88|3.98|0.39|3.50|4.71|0|90.00|2.56|82.97|18.97|43.67|1.42|
|MazeE1-v0|28.13|1.52|4.14|1.48|3.33|9.79|0|93.54|1.11|4048.50|219.68|869.87|54.00|

### Complexity and Optimal Performance by Maze

These measures come from Zatuchna and Bagnall paper :

> Zhanna V. Zatuchna and Anthony Bagnall. 2009. Learning Mazes with Aliasing States: An LCS Algorithm with Associative Perception. Adaptive Behavior - Animals, Animats, Software Agents, Robots, Adaptive Systems 17, 1 (February 2009), 28-57. DOI=http://dx.doi.org/10.1177/1059712308099230

$\phi$ is about the average distance to reward in maze. Its calculation depends mainly on the type of maze. Here is reported $\phi$' if the maze is aliased, the original $\phi$ otherwise.

$\psi$ is about the complexity of the maze that depends on the average distance to reward and on the average steps taken by trained Q-learning.

The question marks (**?**) highlight some discrepancies present in the original paper.

The asterisks (**\***) indicate pieces of information were not provided.

For further details, please see the original paper.

#### For Type III Aliased Mazes

|       |MazeE2      |Woods101demi|Maze10      |Woods102    |Woods100    |Woods101    |MazeE1       |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:-----------:|
|$\phi$'|2.33        |3.1         |5.17        |3.31        |2.33        |2.9         |3.07         |
|$\psi$ |251.2       |251 **?**   |171         |167         |166         |149         |167 **?**    |

#### For Type II Aliased Mazes

|       |MazeF4      |Maze7       |MiyazakiB   |
|-------|:----------:|:----------:|:----------:|
|$\phi$'|4.5 **?**   |4.33        |3.33        |
|$\psi$ |47 **?**    |82 **?**    |1.03        |

#### For Type I Aliased Mazes

|       |MazeB       |Littman89   |MiyazakiA   |MazeD       |Cassandra4x4|Littman57   |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|
|$\phi$'|3.5         |3.77        |3.05        |2.75        |2.27        |3.71        |
|$\psi$ |1.26        |61 **?**    |69 **?**    |1.03        |1           |154 **?**   |

#### For Non Aliased Mazes

|       |Maze4       |Maze5       |MazeA       |MazeF1      |MazeF2      |MazeF3      |Woods1      |Woods14     |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|
|$\phi$ |3.5         |4.61        |4.23        |1.8         |2.5         |3.38        |1.63        |9.5         |
|$\psi$ | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     |